In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4880%2F7569%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240724%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240724T112211Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D96d58db971b27bcf0ac82c42fd42750ab0dc02a1fae3a67dee999a303bb02a7e8b3dbbc7ba51eb4e728cd46087ff2c6557d7b1e499c309c4aeb76b97dd71b2174cb7f0ec6cf2d5f02d5ddc449bcbd41a9fd2da4503738e661c85d6cdbd02a0c0b477313ece8abadb774959c09d05a648f1b47527813220ef3265c2595ed70031a4ced95c4d8a1b9ec007b2b84eb292b91ee1794526c127c655b8856db02f312fe62a6b48c2b3e769b85102bcf59b2cbd2cb099000c5f2397baf90b8478b075853b2de3344e47135d455739b508e8a95b8d4b4942300905b71cde49cf732dbe0c3fe5959672738dcb5215f7e337665b9fb76e32e765e983886bafbf1b71b62bae'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 22617265 bytes downloaded
Downloaded and uncompressed: 
Data source import complete.


### Hu Moments for Image Recognition: Intro

Moments are very common features extracted from images to be used in pattern recogntion tasks, such as face recognition and shape retrieval. In this short notebook, let's take a look at Hu Moments, undoubtedly the most important work of this domain. The main reference is the work **Visual Pattern Recognition by Moment Invariants** wrote by MK Hu in [1962](https://ieeexplore.ieee.org/document/1057692). Both the math and Python code are provided to better understanding of the subject, alongside more detailed explanations. Please feel free to help, share and improve this document.

### Raw Moments

We define _Image Moments_ as **the weighted average of all pixel intensities of a image**. Consider a binary image described by the function $I(x, y)$ with dimenisons $NxN$ pixels (any size is possible, not just square ones), we can calculate the raw moments using:

$$M_{pq} = \sum_{x=0}^{N-1}\sum_{y=0}^{N-1}x^py^qI(x, y)$$

The above expression shows a summation of all pixel intensities pondered by its location $(x, y)$ over the powers $p$ and $q$. In other words, image moments are values that carry both spatial and intensity information, e.g, **shape**. $p$ and $q$ are the weights of the horizontal and vertical dimensions, respectivelly. The sum $p+q$ is the _moment order_.

### Centroid Localization

We can use the raw moments to extract important information of an image. By doing $M_{00}$, we are accumulating the non-zero intensities. It's like describing the spatial information of the pixel "blob". Similary, doing for the $X$ and $Y$ dimensions ($M_{10}$ and $M_{01}$), one can pinpoint the centroid coordinates $(\bar{x}, \bar{y})$ of the blob by doing:

$$\bar{x} = \frac{M_{10}}{M_{00}}$$
$$\bar{y} = \frac{M_{01}}{M_{00}}$$

### Translation Invariance

The centroid can be used to rewrite the raw moment equation to achieve the **translation invariant** momento $\mu_{pq}$:

$$\mu_{pq} = \sum_{x=0}^{N-1}\sum_{y=0}^{N-1}(x - \bar{x})^p(y - \bar{y})^qI(x, y)$$

Now, the relative spatial information of the centroid is being take in consideration, so no matter where the blob is localized the moments will be (roughly) the same.

### Scale Invariance

Scaling (change of size) is another very common transformation performed in images. This scaling can be uniform (the same in both dimensions) or non-uniform. Hu showed that you can relate the zero order translate invariant moment to get scale invariants $\eta_{pq}$:

$$\eta_{pq} = \frac{\mu_{pq}}{\mu_{00}^{1 + \frac{p+q}{2}}}$$

### Hu Moments

We call **Hu Moments** the set of 7 values propesed by Hu in his 1962 work _Visual Pattern Recognition by Moment Invariants_:

$h_1 = \eta_{20} + \eta_{02}$

$h_2 = (\eta_{20} - \eta_{02})^2 + 4(\eta_{11})^2$

$h_3 = (\eta_{30} - 3\eta_{12})^2 + 3(\eta_{03} - 3\eta_{21})^2$

$h_4 = (\eta_{30} + \eta_{12})^2 + (\eta_{03} + \eta_{21})^2$

$h_5 = (\eta_{30} - 3\eta_{12})(\eta_{30} + \eta_{12})[(\eta_{30} + \eta_{12})^2 - 3(\eta_{03} + \eta_{21})^2] + (3\eta_{21} - \eta_{03})(\eta_{03} + \eta_{21})[3(\eta_{30} + \eta_{12})^2 - (\eta_{03} + \eta_{21})^2]$

$h_6 = (\eta_{20} - \eta_{02})[(\eta_{30} + \eta_{12})^2 - 7(\eta_{03} + \eta_{21})^2] + 4\eta_{11}(\eta_{30} + \eta_{12})(\eta_{03} + \eta_{21})$

$h_7 = (3\eta_{21} - \eta_{03})(\eta_{30} + \eta_{12})[(\eta_{30} + \eta_{12})^2 - 3(\eta_{03} + \eta_{21})^2] + (\eta_{30} - 3\eta_{12})(\eta_{03} + \eta_{21})[3(\eta_{30} + \eta_{12})^2 - (\eta_{03} + \eta_{21})^2]$

### The Four Shapes Dataset


Para ilustrar o uso de Momentos Hu no reconhecimento de padrões, usaremos o conjunto de dados Four Shapes. Este conjunto de dados contém 14.970 amostras de quatro classes: círculos, quadrados, estrelas e triângulos. Você pode obter uma cópia gratuita dos dados no [Kaggle](https://www.kaggle.com/smeschke/four-shapes). Criamos as animações abaixo usando as primeiras 400 amostras de cada classe para mostrar a diversidade de exemplos:


|                Circles               |                Squares               |                 Stars                |               Triangles              |
|:------------------------------------:|:------------------------------------:|:------------------------------------:|:------------------------------------:|
| ![](https://i.imgur.com/XcY2ehb.gif) | ![](https://i.imgur.com/hDzvMYj.gif) | ![](https://i.imgur.com/FRE8Pl1.gif) | ![](https://i.imgur.com/ThGXvP8.gif) |

### Código

importando ```opencv-python```, ```scikit-learn```, ```numpy``` and ```pandas```.

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.svm import SVC as SVM
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestCentroid as NC
from sklearn.model_selection import train_test_split as data_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

Criando dois dicionarios: um com as classes das amostras, e outro com os classificadores.

In [3]:
classes = {
	"circle"   : 1,
	"square"   : 2,
	"star"     : 3,
	"triangle" : 4
}

classifiers = {
	"LDA"        : LDA(),
	"QDA"        : QDA(),
	"SVM_linear" : SVM(kernel="linear"),
	"SVM_radial" : SVM(kernel="rbf")
}

A primeira função que vamos criar é responsável por extrair os momentos de todas as amostras, armazenar o rótulo da classe em oito colunas e salvar tudo em um arquivo CSV. Além disso, uma simples transformação logarítmica é realizada para equalizar as ordens dos momentos, o que é uma boa etapa de pré-processamento ao lidar com este tipo de recurso.

In [4]:
def feature_extraction(data_file, dataset="../input/shapes/"):
	dump = []

	print("Extracting Hu moments...")

	for c, idx in classes.items():
		class_folder = dataset + "{}/".format(c)

		for f in os.listdir(class_folder):
			fpath = class_folder + f
			sample = int(f.replace(".png", ""))

			img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)
			img = cv2.bitwise_not(img)
			hu = cv2.HuMoments(cv2.moments(img))

			for i in range(0, 7):
				hu[i] = -1 * np.sign(hu[i]) * np.log10(np.abs(hu[i]))

			hu = hu.reshape((1, 7)).tolist()[0] + [sample, idx]
			dump.append(hu)

		print(c, "ok!")

	cols = ["hu1", "hu2", "hu3", "hu4", "hu5", "hu6", "hu7", "sample", "class"]

	df = pd.DataFrame(dump, columns=cols)
	df.to_csv(data_file, index=None)

	print("Extraction done!")

By executing this function and asking to store the results in ```hu_moments.csv```, we have:

In [5]:
data_file = "hu_moments.csv"

feature_extraction(data_file)

Extracting Hu moments...
circle ok!
square ok!
star ok!
triangle ok!
Extraction done!


The next function performs the classification. The feature vectors were split in training and test sets with 70/30 proportion, respectively, with a default number of test iterations set to 100. This function returns a dataframe with the results of each round.

In [6]:
def classification(data_file, rounds=100, remove_disperse=[]):
	df = pd.read_csv(data_file)
	df = df.drop(["sample"], axis=1)

	if remove_disperse:
		df = df.drop(remove_disperse, axis=1)

	X = df.drop(["class"], axis=1)
	y = df["class"]

	ans = {key: {"score" : []} for key, value in classifiers.items()}

	print("Classifying...")

	for i in range(rounds):
		X_train, X_test, y_train, y_test = data_split(X, y, test_size=0.3)

		for name, classifier in classifiers.items():
			scaler = StandardScaler()
			scaler.fit(X_train)
			X_train = scaler.transform(X_train)
			X_test = scaler.transform(X_test)

			classifier.fit(X_train, y_train)
			score = classifier.score(X_test, y_test)

			ans[name]["score"].append(score)

	print("Classification done!")

	return ans

The next line executes it and stores the result in ```ans```:

In [7]:
ans = classification(data_file)

Classifying...
Classification done!


Let's visualize the classification performance in a more frendly way using the ```summary()``` function:

In [8]:
def sumary(ans, title="Summary"):
	size = 70
	separator = "-"

	print(separator*size)
	print("SUMARY: {}".format(title))
	print(separator*size)
	print("CLASSIF\t\tMEAN\tMEDIAN\tMINV\tMAXV\tSTD")
	print(separator*size)

	for n in ans:
		m = round(np.mean(ans[n]["score"])*100, 2)
		med = round(np.median(ans[n]["score"])*100, 2)
		minv = round(np.min(ans[n]["score"])*100, 2)
		maxv = round(np.max(ans[n]["score"])*100, 2)
		std = round(np.std(ans[n]["score"])*100, 2)

		print("{:<16}{}\t{}\t{}\t{}\t{}".format(n, m, med, minv, maxv, std))

	print(separator*size)
	print()

In [9]:
sumary(ans)

----------------------------------------------------------------------
SUMARY: Summary
----------------------------------------------------------------------
CLASSIF		MEAN	MEDIAN	MINV	MAXV	STD
----------------------------------------------------------------------
LDA             99.49	99.49	99.2	99.73	0.1
QDA             99.84	99.84	99.71	99.98	0.05
SVM_linear      99.92	99.93	99.8	100.0	0.04
SVM_radial      99.98	99.98	99.89	100.0	0.02
----------------------------------------------------------------------



## Conclusão

Como podemos ver, todos os classificadores tiveram um desempenho quase perfeito nesta tarefa. O classificador SVM configurado com Linear Kernel apresentou a maior precisão média e o menor desvio padrão, tornando-o a melhor escolha nesta aplicação simples. Com matemática simples envolvida e um tempo de extração/classificação bastante rápido, os Momentos Hu são uma boa referência para reconhecimento de padrões visuais, bem como um bom exemplo de entrada para aqueles que estão começando com visão computacional.
